In [1]:
import time
import torch

from rcpsp_inst import load_dataset
from net import Net
from aco import ACO_RCPSP

lr = 1e-3
EPS = 1e-10
ALPHA = 0.05
T=5
device = 'cpu'

In [2]:
@torch.no_grad()
def infer_instance(model, rcpsp, n_ants, t_aco_diff):
    if model:
        model.eval()
        pyg_data = rcpsp.to_pyg_data()
        phe_vec, heu_vec = model(pyg_data, require_phe=True, require_heu=True)
        phe_mat, heu_mat = model.reshape(pyg_data, phe_vec) + EPS, model.reshape(pyg_data, heu_vec) + EPS
    
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            pheromone=phe_mat,
            heuristic=heu_mat,
            device=device
        )
    
    else:
        aco = ACO_RCPSP(
            rcpsp,
            n_ants=n_ants,
            device=device
        )
        
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t).cost
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for instance in dataset:
        results = infer_instance(model, instance, n_ants, t_aco_diff)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on j30

In [3]:
n_ants = 5
n_node = 30
t_aco = [1, 10, 20, 30, 40, 50, 100]
_, test_list = load_dataset("../data/rcpsp/j30rcp")
# test_list = test_list[:50]

MetaACO

In [4]:
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/rcpsp/rcpsp{n_node}-4.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  80.80995345115662
T=1, average cost is 65.87000274658203.
T=10, average cost is 60.790000915527344.
T=20, average cost is 59.970001220703125.
T=30, average cost is 59.52000045776367.
T=40, average cost is 59.13999938964844.
T=50, average cost is 58.869998931884766.
T=100, average cost is 58.310001373291016.


ACO

In [5]:
avg_aco_best, duration = test(test_list, None, n_ants, t_aco)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))

total duration:  83.8741614818573
T=1, average cost is 67.87999725341797.
T=10, average cost is 62.43000030517578.
T=20, average cost is 61.459999084472656.
T=30, average cost is 61.0.
T=40, average cost is 60.66999816894531.
T=50, average cost is 60.619998931884766.
T=100, average cost is 60.369998931884766.
